
# Slowly Changing Dimensions (SCD) – Type 0 and Type 1



## What is a Slowly Changing Dimension (SCD)?

A **Slowly Changing Dimension** is a dimension table where attribute values change **infrequently over time**.

Unlike fact tables, dimension tables represent descriptive information such as:
- Customer profile
- Product details
- Employee attributes

Handling changes in dimension attributes is a **core data modeling decision** in data warehousing.



## SCD Type 0 – No Change (Immutable Attributes)

### Concept

- Attribute values **never change after insertion**
- Updates from source systems are **ignored**
- Used for attributes that must remain historically accurate

### Typical SCD Type 0 Attributes
- Date of Birth
- National ID
- Original joining date
- Government-issued identifiers



## Environment Setup (MySQL)

We begin by creating a dedicated database for SCD demonstrations.


In [ ]:

CREATE DATABASE IF NOT EXISTS dw_scd_demo;
USE dw_scd_demo;



## Creating the Customer Dimension Table

This table contains both:
- **Immutable attributes** (SCD Type 0)
- **Mutable attributes** (SCD Type 1)

This mirrors real-world data warehouse design.


In [ ]:

CREATE TABLE dim_customer (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(100),
    date_of_birth DATE,          -- SCD Type 0
    city VARCHAR(50),            -- SCD Type 1
    email VARCHAR(100)           -- SCD Type 1
);



## Loading Initial Customer Data

This represents the **first load** from source systems into the data warehouse.


In [ ]:

INSERT INTO dim_customer (customer_id, customer_name, date_of_birth, city, email) VALUES
(1, 'Ravi Kumar', '1990-05-12', 'Bengaluru', 'ravi.kumar@email.com'),
(2, 'Anita Sharma', '1988-11-03', 'Mumbai', 'anita.sharma@email.com');



## Verifying Initial Load


In [ ]:

SELECT * FROM dim_customer;



## Demonstrating SCD Type 0 (Date of Birth)

### Scenario

Source system sends a correction:
- Ravi Kumar's **Date of Birth** is sent as `1991-05-12` instead of `1990-05-12`

**Business Rule:**  
Date of Birth is treated as an **immutable attribute** in the data warehouse.

➡️ **The update must be ignored**.



### Why We Ignore This Change

- Date of Birth should not change for analytical consistency
- Accepting such updates can break historical reports
- Corrections must be handled **outside the warehouse**, not via overwrites


In [ ]:

-- ❌ SCD Type 0: This update must NOT be applied
-- UPDATE dim_customer
-- SET date_of_birth = '1991-05-12'
-- WHERE customer_id = 1;



### Result After SCD Type 0 Handling

The Date of Birth remains unchanged in the dimension table.


In [ ]:

SELECT customer_id, customer_name, date_of_birth
FROM dim_customer;



## SCD Type 1 – Overwrite (City and Email)

### Concept

- Attribute values are **overwritten**
- No historical tracking
- Dimension always reflects the **latest known state**

### Typical SCD Type 1 Attributes
- Address
- City
- Email
- Phone number



## Demonstrating SCD Type 1

### Scenario

Source system sends updated information:
- Ravi Kumar moved from **Bengaluru to Chennai**
- Email address changed

For SCD Type 1, we **overwrite the existing values**.


In [ ]:

UPDATE dim_customer
SET city = 'Chennai',
    email = 'ravi.kumar@newmail.com'
WHERE customer_id = 1;



## Verifying Data After SCD Type 1 Update

Notice:
- Date of Birth is unchanged (SCD 0)
- City and Email are overwritten (SCD 1)


In [ ]:

SELECT * FROM dim_customer;



## SCD Type 0 vs SCD Type 1 – Comparison

| Aspect | SCD Type 0 | SCD Type 1 |
|------|-----------|-----------|
| Attribute Nature | Immutable | Mutable |
| Update Allowed | ❌ No | ✅ Yes |
| History Stored | Implicit | No |
| Example | Date of Birth | City, Email |



## Key Takeaways

- **SCD Type 0** protects critical attributes from change
- **SCD Type 1** keeps dimensions up-to-date
- Both types are often used **together in the same table**
- Choosing the correct SCD type is a **business decision**, not a technical one